In [1]:
# Task 3
# Import Package
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

Import your csv or excel file (Adjust the code accordingly). Rename the columns for easier access. Take a look at your dataframe and understand the meaning of each column.

In [2]:
# Import Data
measure = pd.read_csv("./Marker diameter.csv", usecols=np.arange(0,4))
measure.rename(columns={"Reading (unit = mm)" : "Reading", "Marker" : "Part"}, inplace=True)
measure

,Run Order,Part,Operator,Reading
0,1,4,1,16.97
1,2,3,3,17.24
2,3,4,2,16.98
3,4,2,2,16.99
4,5,2,3,17.18
5,6,2,1,17.12
6,7,3,1,17.15
7,8,3,2,17.05
8,9,2,3,17.16
9,10,4,3,17.04


In [3]:
# Calculate Rbar and xbb
oplist = [1, 2, 3]
partlist = [1, 2, 3, 4]
for op in oplist:
    Op = measure[measure.Operator == op]
    R = []
    xbar = []
    for part in partlist:
        data = Op.loc[Op.Part == part, "Reading"]
        R.append(max(data) - min(data))
        xbar.append(np.mean(data))
    print("Operator %s Rbar: %.4f / xbb: %.4f" % (op, np.mean(R), np.mean(xbar)))


Operator 1 Rbar: 0.0100 / xbb: 17.0525
Operator 2 Rbar: 0.0400 / xbb: 16.9600
Operator 3 Rbar: 0.0475 / xbb: 17.1288


In [4]:
Rbb = (0.01 + 0.04 + 0.0475)/3 # Copy the result from above output
Rxbb = 17.1288 - 16.9600 # Find the max Rbar and min Rbar
d2_n = 1.128 # n=2
d2_o = 1.693

# Calculate sigma
sigma_repeat = Rbb/d2_n
sigma_reproduce = Rxbb/d2_o
sigma_gauge = np.sqrt(sigma_repeat**2 + sigma_reproduce**2)
sigma_total = np.std(measure["Reading"], ddof=1)
sigma_part = np.sqrt(sigma_total**2 - sigma_gauge**2)

ResultTable = pd.DataFrame(columns=["Sigma", "Variance", "Ratio"])
ResultTable.loc[len(ResultTable)] = [sigma_gauge, sigma_gauge**2, (sigma_gauge**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_repeat, sigma_repeat**2, (sigma_repeat**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_reproduce, sigma_reproduce**2, (sigma_reproduce**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_part, sigma_part**2, (sigma_part**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_total, sigma_total**2, (sigma_total**2/sigma_total**2)*100]
ResultTable.index = ["sigma_gauge", "sigma_repeat", "sigma_reproduce", "sigma_part", "sigma_total"]
print(ResultTable)

                    Sigma  Variance       Ratio
sigma_gauge      0.103784  0.010771   87.571656
sigma_repeat     0.028812  0.000830    6.749161
sigma_reproduce  0.099705  0.009941   80.822495
sigma_part       0.039098  0.001529   12.428344
sigma_total      0.110905  0.012300  100.000000


In [5]:
# ANOVA - Full Model
model = ols('Reading ~ C(Part) + C(Operator) + C(Part):C(Operator)', data=measure).fit()
AnovaTable = sm.stats.anova_lm(model, type=2)
print(AnovaTable)

                       df    sum_sq   mean_sq          F    PR(>F)
C(Part)               3.0  0.146312  0.048771  36.015385  0.000003
C(Operator)           2.0  0.114258  0.057129  42.187692  0.000004
C(Part):C(Operator)   6.0  0.006075  0.001012   0.747692  0.622797
Residual             12.0  0.016250  0.001354        NaN       NaN


In [9]:
# Define p, o, n
p = 4 # 4 makers
o = 3 # 3 operators
n = 2 # 2 measurements

# Copy MS from ANOVA table
MSp = 4.777778
MSo = 0.194444
MSpo = 0.194444
MSe =  0.027778


# Calculate sigma
sigma_part = np.sqrt((MSp - MSpo)/(o*n))
sigma_o = np.sqrt((MSo - MSpo)/(p*n))
sigma_po = np.sqrt((MSpo - MSe)/(n))
sigma_repeat = np.sqrt(MSe)
sigma_reproduce = np.sqrt(sigma_po**2 + sigma_o**2)
sigma_gauge = np.sqrt(sigma_repeat**2 + sigma_reproduce**2)
sigma_total = np.sqrt(sigma_gauge**2 + sigma_part**2)

ResultTable = pd.DataFrame(columns=["Sigma", "Variance", "Ratio"])
ResultTable.loc[len(ResultTable)] = [sigma_gauge, sigma_gauge**2, (sigma_gauge**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_repeat, sigma_repeat**2, (sigma_repeat**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_reproduce, sigma_reproduce**2, (sigma_reproduce**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_part, sigma_part**2, (sigma_part**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_total, sigma_total**2, (sigma_total**2/sigma_total**2)*100]
ResultTable.index = ["sigma_gauge", "sigma_repeat", "sigma_reproduce", "sigma_part", "sigma_total"]
ResultTable

,Sigma,Variance,Ratio
sigma_gauge,0.333333,0.111111,12.698400
sigma_repeat,0.166667,0.027778,3.174629
sigma_reproduce,0.288675,0.083333,9.523771
sigma_part,0.874007,0.763889,87.301600
sigma_total,0.935414,0.875000,100.000000


Use reduce model if the interaction term is not significant. (p-value > $\alpha$)

In [7]:
# Anova - Reduce Model
model = ols('Reading ~ C(Part) + C(Operator)', data=measure).fit()
AnovaTable = sm.stats.anova_lm(model, type=2)
print(AnovaTable)

               df    sum_sq   mean_sq          F        PR(>F)
C(Part)       3.0  0.146312  0.048771  39.322508  4.130536e-08
C(Operator)   2.0  0.114258  0.057129  46.061590  8.328060e-08
Residual     18.0  0.022325  0.001240        NaN           NaN


In [10]:
# Define p, o, n
p = 4 # 4 makers
o = 3 # 3 operators
n = 2 # 2 measurements

# Copy MS from ANOVA table
MSp = 4.777778
MSo = 0.194444
MSe =  0.061111


# Calculate sigma
sigma_part = np.sqrt((MSp - MSe)/(o*n))
sigma_reproduce = np.sqrt((MSo - MSe)/(p*n))
sigma_repeat = np.sqrt(MSe)
sigma_gauge = np.sqrt(sigma_repeat**2 + sigma_reproduce**2)
sigma_total = np.sqrt(sigma_gauge**2 + sigma_part**2)

ResultTable = pd.DataFrame(columns=["Sigma", "Variance", "Ratio"])
ResultTable.loc[len(ResultTable)] = [sigma_gauge, sigma_gauge**2, (sigma_gauge**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_repeat, sigma_repeat**2, (sigma_repeat**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_reproduce, sigma_reproduce**2, (sigma_reproduce**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_part, sigma_part**2, (sigma_part**2/sigma_total**2)*100]
ResultTable.loc[len(ResultTable)] = [sigma_total, sigma_total**2, (sigma_total**2/sigma_total**2)*100]
ResultTable.index = ["sigma_gauge", "sigma_repeat", "sigma_reproduce", "sigma_part", "sigma_total"]



ResultTable

,Sigma,Variance,Ratio
sigma_gauge,0.278886,0.077778,9.003199
sigma_repeat,0.247206,0.061111,7.073943
sigma_reproduce,0.129099,0.016667,1.929256
sigma_part,0.886629,0.786111,90.996801
sigma_total,0.929456,0.863889,100.000000
